In [ ]:
import numpy as np
import pandas as pd
import dateutil.parser


import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

import requests
import json

In [ ]:
#Helsingin Sanomat open data for comparison
url="https://w3qa5ydb4l.execute-api.eu-west-1.amazonaws.com/prod/finnishCoronaData"
res=requests.get(url)
hs_data = json.loads(res.content)
hs_df=pd.io.json.json_normalize(hs_data['confirmed'])
hs_df['one']=1
hs_df['total'] = hs_df.one.cumsum()

In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt

start = dateutil.parser.parse("2020-02-24")

# Total population, N.
N = 5.5e6
# Initial number of infected and recovered individuals, I0 and R0.
I0, R0, D0 = 3, 0, 0
# Everyone else, S0, is susceptible to infection initially.
S0 = N - I0 - R0
# Contact rate, beta, and mean recovery rate, gamma, (in 1/days).
beta, gamma, delta = 0.3, 1./14, 0.037/14.0
# A grid of time points (in days)
t = np.linspace(0, 500, 500)
R0=4.2
R0_after=2.4
day=60

death_prob_per_day = 0.037/contagious_days
death_prob_ic = 0.15/contagious_days
death_prob_no_care = 0.3/contagious_days
ic_need = 0.06
ic_capacity=1000.

def betaT(t):
    r0_default = 3.0
    r0 = r0_default
    if t<30.:
        r0 = r0_default
    if t<37:
        r0 = r0_default * 0.5
    if t < 42:
        r0 = r0_default * 0.75

    return r0*gamma

def deltaI(i):
    if i*ic_need <  ic_capacity:
        return death_prob_per_day
    if i*ic_need > ic_capacity:
        return 

# The SIR model differential equations.
def deriv(y, t, N, beta, gamma):
    S, I, R, D = y
    beta = betaT(t)
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I - delta * I
    dRdt = gamma * ( I - delta * I)
    
    if I*ic_need < ic_capacity:
        dDdt = (death_prob_per_day * I * (1.0-ic_need) 
                + death_prob_ic * ic_need * I
               )
    else:
        dDdt = (death_prob_per_day * I * (1.0-ic_need) 
        + death_prob_ic * ic_capacity
        + death_prob_no_care * (I*ic_need - ic_capacity)
        )
    return dSdt, dIdt, dRdt, dDdt

# Initial conditions vector
y0 = S0, I0, R0, D0
# Integrate the SIR equations over the time grid, t.
ret = odeint(deriv, y0, t, args=(N, beta, gamma))
S, I, R, D = ret.T

df2=pd.DataFrame()
df2['t'] = t
df2['S'] = S
df2['I'] = I
df2['R'] = R
df2['D'] = D
df2['ts'] = pd.to_datetime(start.timestamp() + df2.t*86400, unit='s')
I_sc1 = I
data=[
      #go.Scatter(x=t, y=S, name="Suspectible"),
      go.Scatter(x=df2.ts, y=df2.I, name="Infected"),
      go.Scatter(x=df2.ts, y=df2.R, name="Recovered with immunity"),
      #go.Scatter(x=df2.ts, y=df2.D, name="Dead"),
      go.Scatter(x=df2.ts, y=df2.t, name="Day"),
    
     ]
layout = go.Layout(
    title='Covid19, SIRD model' ,
    xaxis=dict(
        title='day',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    ),
    yaxis=dict(
        #type='log',
        title='people',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        ),
        #range = [1,500000],
    )
)
iplot({ 'data': data, "layout": layout }) 

In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt

start = dateutil.parser.parse("2020-02-24")

# Total population, N.
N = 5.5e6
# Initial number of infected and recovered individuals, I0 and R0.
I0, R0, D0 = 3, 0, 0
# Everyone else, S0, is susceptible to infection initially.
S0 = N - I0 - R0
# Contact rate, beta, and mean recovery rate, gamma, (in 1/days).
beta, gamma, delta = 0.3, 1./14, 0.037/14.0
# A grid of time points (in days)
t = np.linspace(0, 500, 500)
R0=4.2
R0_after=2.4
day=60

death_prob_per_day = 0.037/contagious_days
death_prob_ic = 0.15/contagious_days
death_prob_no_care = 0.3/contagious_days
ic_need = 0.06
ic_capacity=1000.

def betaT(t):
    r0_default = 3.0
    r0 = r0_default
    if t<30.:
        r0 = r0_default
    if t<37:
        r0 = r0_default * 0.75
    if t < 42:
        r0 = r0_default * 0.5

    return r0*gamma

def deltaI(i):
    if i*ic_need <  ic_capacity:
        return death_prob_per_day
    if i*ic_need > ic_capacity:
        return 

# The SIR model differential equations.
def deriv(y, t, N, beta, gamma):
    S, I, R, D = y
    beta = betaT(t)
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I - delta * I
    dRdt = gamma * ( I - delta * I)
    
    if I*ic_need < ic_capacity:
        dDdt = (death_prob_per_day * I * (1.0-ic_need) 
                + death_prob_ic * ic_need * I
               )
    else:
        dDdt = (death_prob_per_day * I * (1.0-ic_need) 
        + death_prob_ic * ic_capacity
        + death_prob_no_care * (I*ic_need - ic_capacity)
        )
    return dSdt, dIdt, dRdt, dDdt

# Initial conditions vector
y0 = S0, I0, R0, D0
# Integrate the SIR equations over the time grid, t.
ret = odeint(deriv, y0, t, args=(N, beta, gamma))
S, I, R, D = ret.T

df2=pd.DataFrame()
df2['t'] = t
df2['S'] = S
df2['I'] = I
I_sc2 = I
df2['R'] = R
df2['D'] = D
df2['ts'] = pd.to_datetime(start.timestamp() + df2.t*86400, unit='s')
data=[
      #go.Scatter(x=t, y=S, name="Suspectible"),
      go.Scatter(x=df2.ts, y=df2.I, name="Infected"),
      go.Scatter(x=df2.ts, y=df2.R, name="Recovered with immunity"),
      #go.Scatter(x=df2.ts, y=df2.D, name="Dead"),
      go.Scatter(x=df2.ts, y=df2.t, name="Day"),
    
     ]

iplot({ 'data': data, "layout": layout }) 

In [ ]:
data=[
      #go.Scatter(x=t, y=S, name="Suspectible"),
      go.Scatter(x=df2.t, y=I_sc1, name="Infected1"),
      go.Scatter(x=df2.t, y=I_sc2, name="Infected2"),
      #go.Scatter(x=df2.ts, y=df2.t, name="Day"),
    
     ]

iplot({ 'data': data, "layout": layout }) 

In [ ]:
df2